
# Simulación de una variable aleatoria Binomial usando la pseudoinversa de la CDF

Este cuaderno implementa el algoritmo de simulación de una variable aleatoria
Binomial \(X \sim \mathrm{Bin}(n,p)\) a partir de un generador de números aleatorios
uniformes en el intervalo (0,1].

La idea principal es:

1. Generar un número uniforme \(u \in (0,1]\.\)
2. Construir la distribución acumulada de la Binomial:
   \(F(k) = \sum_{j=0}^{k} \binom{n}{j} p^{j} (1-p)^{n-j}\).
3. Encontrar el menor entero \(k\) tal que \(F(k) \geq u\).
4. Devolver ese \(k\) como una realización de la variable Binomial.


In [ ]:

import math
import random
from typing import List

import matplotlib.pyplot as plt


In [ ]:

def binomial_pmf(n: int, p: float, k: int) -> float:
    """Calcula P(X = k) para X ~ Bin(n, p)."""
    if k < 0 or k > n:
        return 0.0
    coef = math.comb(n, k)
    return coef * (p ** k) * ((1 - p) ** (n - k))


def binomial_inverse_cdf_sample(n: int, p: float) -> int:
    """
    Genera una muestra de X ~ Bin(n, p) usando la pseudoinversa de la CDF.
    """
    u = random.random()
    if u == 0.0:
        u = 1e-12

    F = 0.0
    for k in range(0, n + 1):
        F += binomial_pmf(n, p, k)
        if u <= F:
            return k
    return n


In [ ]:

def sample_binomial(n: int, p: float, N: int) -> List[int]:
    """Genera N muestras independientes de X ~ Bin(n, p)."""
    muestras = []
    for _ in range(N):
        x = binomial_inverse_cdf_sample(n, p)
        muestras.append(x)
    return muestras


In [ ]:

# Parámetros de la Binomial
n = 5
p = 0.3
N = 1000  # número de simulaciones

datos = sample_binomial(n, p, N)

print(f"Primeras 20 observaciones simuladas de X ~ Bin({n}, {p}):\n", datos[:20])


In [ ]:

# Cálculo de frecuencias relativas
valores = list(range(n + 1))
conteos = [datos.count(k) for k in valores]
frec_rel = [c / N for c in conteos]

# Probabilidades teóricas
probs_teoricas = [binomial_pmf(n, p, k) for k in valores]

print("k | frec. relativa | prob. teórica")
for k, fr, pt in zip(valores, frec_rel, probs_teoricas):
    print(f"{k:1d} | {fr:13.4f} | {pt:13.4f}")

plt.figure()
plt.bar(valores, frec_rel, alpha=0.6, label="Frecuencia relativa")
plt.plot(valores, probs_teoricas, marker="o", linestyle="--", label="Probabilidad teórica")
plt.xlabel("k")
plt.ylabel("Probabilidad / Frecuencia relativa")
plt.title(f"Comparación: Binomial(n={n}, p={p})")
plt.legend()
plt.show()
